# Exercise 5
(v.1.4)
## Lecture 9 & 10 - Reinforcement Learning

### Learning Goals
After successfully completing this assignment, you should be able to:
- explain why exploration and exploitation are important concepts in reinforcement learning problems
- understand better the Multi-Armed Bandit problem
- implement a strategy for maximizing the cumulative reward (UCB)
- understand the Frozen Lake game/problem
- implement an algorithm for training an agent to find an optimal path through this grid game (Q-learning)
- understand on an intuitive level how different parameters impacts the performance

# Table of Contents
1. [Multi-Armed Bandit](#multi_armed_bandit)
2. [Frozen Lake Game](#frozen_lake_game)

# Multi-Armed Bandit <a class="anchor" id="multi_armed_bandit"></a>

![Multi-armed bandit](https://leovan.me/images/cn/2020-05-16-multi-armed-bandit/compulsive-gambling.png "Multi-armed bandit")

Here we will focus on the "Multi-armed bandit" problem. There are 10 one-armed bandit machines where each have different probabilities for winning and with different reward ranges. As we have a limited amount of coins to spend, the goal is to play on these machines in a way that maximizes the cumulative reward. An action at every time step involves selecting and playing on  one of these ten one-armed bandits, or simply arms. For each action, i.e. for each arm that we play, we observe the reward which is a value between 0 and 10 (€). Since we know nothing about the reward properties of these machines to start with, we need to implement a suitable strategy that is based around observing the rewards that we get from playing on these machines.

In [1]:
# Import the libraries we will be using
import pandas as pd
import math
import random
random.seed(1234)
import numpy as np
import tabulate

## Dataset
The dataset that we will use here is `10_armed_bandit_data.csv`. Each row in this dataset shows the reward of playing on each arm at a given time step. With this way of representing the data, the reward from each arm changes for each time step, meaning that also the order in which the arms are selected will influence the outcome. Since there are 1000 rows in the dataset, it means we have 1000 coins to play with.

In [2]:
# Importing the dataset
df = pd.read_csv('10_armed_bandit_data.csv')

# Print the first 5 rows from the dataset.
print(df.head(5))

   Arm 0  Arm 1  Arm 2  Arm 3  Arm 4  Arm 5  Arm 6  Arm 7  Arm 8  Arm 9
0      0      0      0      4      2      0      0      0      0      1
1      0      0      6      0      0      0      0      0      0      0
2      0      0      0      0      3      0      0      0      0      0
3      0      0      7      0      2      0      1      5      0      0
4      0      2      0      5      0      0      0      0      0      0


## Random Selection
In this first part we adopt a simple strategy that involves naively selecting and playing on a random arm each time, independent of the rewards we get.

<div class=" alert alert-warning">
    
### Student Task A5.1

Your task is to implement the Random Selection strategy - selecting a random arm (or action) $a$ at every time step $t$ - by completing the `random_selection()` function. Our budget is 1000 coins.
</div>

In [3]:
# Implementing Random Selection
def random_selection(data):
    
    # There are 10 columns in the dataset, one for each arm/machine.
    # The row count represents the number of times we play, i.e., how many coins we will spend. 
    coins, arms = data.shape  # (1000, 10)
    
    reward_sum_per_arm = [0] * arms
    selection_count_per_arm = [0] * arms
    
    for t in range(0, coins):
        
        ## Select a random arm (action) among the 10 available arms
        # a = random. ...
        
        ## Fetch the reward for selecting the given arm (a) at the given time step (t) from the data
        # reward = data.values[...]
        
        # YOUR CODE HERE
        a = np.random.randint(arms)
        reward = data.values[t, a]
        
        selection_count_per_arm[a] += 1
        reward_sum_per_arm[a] += reward

    return reward_sum_per_arm, selection_count_per_arm


# Run the selection algorithm
reward_sum_per_arm, selection_count_per_arm = random_selection(df)
# Print the outcome and some statistics
print('Total reward using Random Selection: {}'.format(sum(reward_sum_per_arm)))

sorted_indxs = np.argsort([-x for x in selection_count_per_arm])
tab_data = [[indx, selection_count_per_arm[indx]] for indx in sorted_indxs]
headers = ['Arm', 'Times selected']
print('\n' + tabulate.tabulate(tab_data, headers=headers))

Total reward using Random Selection: 820

  Arm    Times selected
-----  ----------------
    7               119
    8               108
    6               107
    0               104
    1               103
    3               103
    9                93
    4                92
    5                91
    2                80


In [4]:
# this cell is for tests


## Upper Confidence Bound (UCB) Selection

Random selection of arms to play on is probably not the optimal approach. To improve our reward we want to use the UCB algorithm to calculate $a_t$ which is the action, or arm, at time $t$ with maximum UCB:
$$a_t = \underset{a\in{A}}{\operatorname{argmax}}\biggr[Q_t(a) + c \sqrt{\frac{\log{t}}{N_t(a)}}\biggr]$$
<br>
Where $Q_t(a)$ is the estimated value of action (arm) $a$ at time step $t$:
$$Q_t(a) = \frac{ \sum_{i=1}^{t-1} R_{i}\mathbf{1}_{A_i = a} }{ \sum_{i=1}^{t-1} \mathbf{1}_{A_i = a} }$$
<br>
$t$ - current time step.<br>
$N_t(a)$ - the number of times that action $a$ has been selected, prior to time $t$.<br>
$c$ - confidence value that controls the level of *exploration* vs *exploitation*. Higher $c$ means higher exploration rate (... of arms that have not been explored much or yielded much reward so far).<br>
$\sum_{i=1}^{t-1} R_{i}\mathbf{1}_{A_i = a}$ - sum of rewards from selecting $a$ up to $t$-1<br>
$\sum_{i=1}^{t-1} \mathbf{1}_{A_i = a}$ - number of times $a$ has been selected up to $t$-1<br>


<div class=" alert alert-warning">
    
### Student Task A5.2

In this assignment your task is to implement the above UCB algorithm, `max_ucb_action()`. As inputs our function takes $c$, $t$, a list storing the accumulated rewards per arm (`reward_sum_per_arm`), and a list with informaion about how many times each arm has been selected so far (`selection_count_per_arm`, this is $\{N_t(A)\}$).
<br><br>
Note, in the beginning the value of $N_t(a)$ will be zero. To avoid the division by zero problem, we can, in those cases, set it to a number that is close to zero, e.g., 1e-100.
</div>

In [5]:
# Example showing how the input arguments might look like at t = 40.
# c = 1
# t = 40
# reward_sum_per_arm = [0, 12, 0, 2, 1, 0, 0, 0, 0, 0]
# selection_count_per_arm = [3, 15, 3, 5, 4, 3, 2, 2, 2, 2]

# reward_sum_per_arm and selection_count_per_arm stores the cummulated values up to the current time step t

def max_ucb_action(c, t, reward_sum_per_arm, selection_count_per_arm):
    arms = len(reward_sum_per_arm)
    confidence_per_arm = [0] * arms
    
    # In this for-loop we calculate the confidence_per_arm (i.e., action a) at the given time step t
    for a in range(0, arms):
        
        Nt = selection_count_per_arm[a]
        if Nt == 0:
            Nt = 1e-100  # To avoid division by zero error
        
        # Qt = ...
        
        ## NB!: use math.log(t+1) to avoid error with math.log(0)
        # Ut = ...  
        
        # confidence_per_arm[a] = ...
        
        
        # YOUR CODE HERE
        
        Qt = reward_sum_per_arm[a] / Nt
        Ut = c * np.sqrt((math.log(t+1)) / (Nt))
        confidence_per_arm[a] = Qt + Ut
    
    
    # Pick the arm/action with the highest upper bound
    max_upper_bound_arm = confidence_per_arm.index(max(confidence_per_arm))
    return max_upper_bound_arm

In [6]:
# this cell is for tests


<div class=" alert alert-warning">
    
### Student Task A5.3

Now that we have implemented `max_ucb_action()`, the next thing to do is to implement the function `ucb_selection()` where we use the UCB selection approach for playing on these 10 one-armed bandits. Our budget is again 1000 coins.
</div>

In [7]:
# Implementing UCB Selection
def ucb_selection(data, c):
    
    ## Implement this in the same way as the above random_selection() function (from A5.1).
    ## The only exception is that you should now use use max_ucb_action() to select arms 
    ## instead of selecting arms at random.
    # ...
    
    # YOUR CODE HERE
    coins, arms = data.shape  # (1000, 10)
    
    reward_sum_per_arm = [0] * arms
    selection_count_per_arm = [0] * arms
    
    for t in range(0, coins):
        
        ## Select a random arm (action) among the 10 available arms
        # a = random. ...
        
        ## Fetch the reward for selecting the given arm (a) at the given time step (t) from the data
        # reward = data.values[...]
        
        # YOUR CODE HERE
        a = max_ucb_action(c, t, reward_sum_per_arm, selection_count_per_arm)
        reward = data.values[t, a]
        
        selection_count_per_arm[a] += 1
        reward_sum_per_arm[a] += reward

    return reward_sum_per_arm, selection_count_per_arm

    
    return reward_sum_per_arm, selection_count_per_arm


# Run the selection algorithm (by default we have set c = 1)
c = 1
reward_sum_per_arm, selection_count_per_arm = ucb_selection(df, c)

# Sanity check
assert(sum(ucb_selection(df, c=1)[0]) == 1200)


# Print the outcome and some statistics
print('Total reward using UCB Selection:', sum(reward_sum_per_arm))

sorted_indxs = np.argsort([-x for x in selection_count_per_arm])
tab_data = [[indx, selection_count_per_arm[indx]] for indx in sorted_indxs]
headers = ['Arm', 'Times selected']
print('\n' + tabulate.tabulate(tab_data, headers=headers))

Total reward using UCB Selection: 1200

  Arm    Times selected
-----  ----------------
    3               874
    9                61
    7                35
    4                 6
    0                 4
    1                 4
    2                 4
    5                 4
    6                 4
    8                 4


In [8]:
# this cell is for tests


<div class=" alert alert-warning">
    
### Bonus Task

So far we have used $c=1$, can you find the $c$ value that provides the highest accumulated reward?
</div>

# The Frozen Lake Game <a class="anchor" id="frozen_lake_game"></a>
<img src="https://debuggirl.files.wordpress.com/2013/03/img_43081.jpg" alt="drawing" width="800"/>

The Frozen Lake game is about safely navigating a frozen lake whose ice happen have several hidden cracks or holes in it, stepping on any of these will result in you falling into the water. The goal is to reach a part of the lake where a hidden treasure is located. Both the weak spots (holes) and the goal are not known beforehand by the player. Here we will train an *agent* to navigate this lake safely and reach the goal. To do so we will be using the **Q-learning** algorithm. Here we use the **Q-value function** $Q({s, a}) = v$ that takes as input the current state $s$ and possible action $a$, and returns its value $v$. The frozen lake is represented as a grid of cells (gridworld), where each cell is associated with a state that can be start `S`, frozen lake `F`, hole in the ice `H`, or goal `G`, and the actions available at each state is to navigate *left*, *down*, *right*, or *up*.<br>
<br>
When playing this game the agent will be iteratively doing the following:
1. Choose an action given the current state of the agent (calculate where to move).
2. Perform the chosen action (move to the chosen cell).
3. Upadete the Q-value function (the Q-table) based on the reward from performing this action.
4. Set the new state of the agent.

In [9]:
# Import the libraries we will be using
import gym
import numpy as np
import time
import random
from IPython.display import clear_output
import os


# Initiate the FrozenLake game
env = gym.make('FrozenLake-v1', is_slippery=False, map_name="4x4")

# Next we render the game board
env.render()


SFFF
FHFH
FFFH
HFFG


After running the above, you should should see the gameboard, i.e. the frozen lake.
<br>
SFFF</br>
FHFH<br>
FFFH<br>
HFFG<br>


It consist of the following cells:
 - S - Start.
 - F - Frozen lake, which is safe to walk on.
 - H - Hole in the ice. Reaching this cell ends the round.
 - G - Goal. Reaching this cell gives +1 point and ends the round.

As mentioned, we will now train an agent to navigate this frozen lake and ultimately reach the goal (`G` cell) with as few steps/turns as possible without falling into the lake (stepping on any of the `H` cells). At start the agent does not know where the goal is or what cells are safe to walk on. However, with repeated trials, combined with the delayed feedback reward gained from stepping on the various cells, the agent should eventually learn to navigate this lake.

In [10]:
state_space_size = env.observation_space.n
action_space_size = env.action_space.n
print('The frozen lake (gameboard) consist of {} states (cells) and {} actions for each of these.'.format(state_space_size, action_space_size))
print('So for each state there are four actions availble:\n0 = ◀️ left\n1 = 🔽 down\n2 = ▶️ right\n3 = 🔼 up')

The frozen lake (gameboard) consist of 16 states (cells) and 4 actions for each of these.
So for each state there are four actions availble:
0 = ◀️ left
1 = 🔽 down
2 = ▶️ right
3 = 🔼 up


## Q-Table
We will be using a **Q-table** as the underlying representation of the value function (Q-value function). For this type of game board, the Q-table is a well suited representation to use. This table will store all the knowledge gained by the agent - each *state* will have information about the *value* of taking any of the available *actions* (◀️, 🔽, ▶️, 🔼) when the policy is to find the hidden treasure without falling into the lake. To start with, all the q-values in this table will be zero, but these will be updated as the agent is playing.

In [11]:
q_table = np.zeros((state_space_size, action_space_size))

# Print some info about it
s = 'Q-table:\n'
i = 0
j = 0
for cr in q_table:
    s += '{}\t<-- state [{},{}]\n'.format(cr, i, j)
    j += 1
    if j >= action_space_size:
        j = 0
        i += 1
print(s)

Q-table:
[0. 0. 0. 0.]	<-- state [0,0]
[0. 0. 0. 0.]	<-- state [0,1]
[0. 0. 0. 0.]	<-- state [0,2]
[0. 0. 0. 0.]	<-- state [0,3]
[0. 0. 0. 0.]	<-- state [1,0]
[0. 0. 0. 0.]	<-- state [1,1]
[0. 0. 0. 0.]	<-- state [1,2]
[0. 0. 0. 0.]	<-- state [1,3]
[0. 0. 0. 0.]	<-- state [2,0]
[0. 0. 0. 0.]	<-- state [2,1]
[0. 0. 0. 0.]	<-- state [2,2]
[0. 0. 0. 0.]	<-- state [2,3]
[0. 0. 0. 0.]	<-- state [3,0]
[0. 0. 0. 0.]	<-- state [3,1]
[0. 0. 0. 0.]	<-- state [3,2]
[0. 0. 0. 0.]	<-- state [3,3]



With an empty Q-table the agent do not know anything about what actions are better than others. In other words, the Q-value function will always return zero for every possible action at each state. Thus in the beginning the agent will instead be taking random actions while exploring the board. This is called *exploration*. However, after having gained some knowledge about the board, the agent can start relying on the experience gained and reflected in the Q-value function. This is called *exploitation*. We will be using the *epsilon greedy strategy* to find a balance between *exploration* (random actions) and *exploitation* (optional actions according to the Q-value function).

In [12]:
# Here we introduce some parameters to use in the Q-training algorithm.

# Learning rate (value between 0 and 1)
learning_rate = 0.1

# Reward discount used in the Bellman equation (value between 0 and 1)
discount_factor = 0.99

## The Bellman Equation
For updating a state in the Q-table we will here be using the following version of the *Bellman equation*:<br>
$$Q^{updated}(s_{t}, a_{t}) := (1-\alpha)Q(s_{t}, a_{t}) + \alpha (r_{t} + \gamma \underset{a}{\max}Q(s_{t+1},a))$$
$s_{t}$ - current state.<br>
$a_{t}$ - chosen action.<br>
$\alpha$ - learning rate.<br>
$r_{t}$ - reward from taking the action $a_{t}$.<br>
$\gamma$ - discount factor.<br>
$s_{t+1}$ - new state after taking action $a_{t}$.<br>
$\underset{a}{\max}Q(s_{t+1},a)$ - the optional value possible from $s_{t+1}$.<br>

<div class=" alert alert-warning">
    
### Student Task A5.4

<strong><i><u>Your task<i></u></strong> is to implement the Bellman equation in `q_update()` that calculates and returns the new value for $Q^{updated}(s_{t}, a_{t})$.<br>It takes as input: Q-table, $s_{t}$, $a_{t}$, $s_{t+1}$, $r_{t}$, $\alpha$, and $\gamma$.
</div>

In [13]:
def q_update(q_table, state, action, new_state, reward, learning_rate, discount_factor):
    # Calculate the new value in the q_table using Bellman equation

    ## Implement the Bellman equation as shown above
    # q_new = q_table[state, action] * ...
    
    # YOUR CODE HERE
    q_new = (1-learning_rate) * q_table[state, action] + learning_rate*(reward + discount_factor*max(q_table[new_state, :]))
    
    return q_new

In [14]:
# this cell is for tests


In [15]:
# For the mentioned epsilon greedy strategy we will need a few more parameters.

# Upper bound for the exploration rate (value between 0 and 1)
max_exploration_rate = 1.0

# Lower bound for the exploration rate (value between 0 and 1)
min_exploration_rate = 0.01

# How much the exploration rate decays per episode (down to the min_exploration_rate, value between 0 and 1).
exploration_decay_rate = 0.001

## Epsilon Greedy Strategy
By using the *epsilon greedy strategy*, we will let the exploration rate start high (1), and then decrease for every episode with exponential decay. The goal here is to find the optimal balance between exploration and exploitation. Each round will consist of 10000 episodes. In the first episodes we want to have a high probility of performing exploration actions to avoid getting stuck in sub-optimal paths (local maximums). However, we want to gradually start leaning more towards exploitation towards the later episodes.<br>
<br>
We will be using the following formula for calculating the exploration rate for a given episode $\epsilon(i)$:<br>
$$\epsilon(i) = \epsilon_{min} + (\epsilon_{max} - \epsilon_{min}) \cdot e^{-\zeta \cdot i}$$
$\epsilon$ - exploration rate.<br>
$i$ - episode number.<br>
$\epsilon_{min}$ - min exploration rate.<br>
$\epsilon_{max}$ - max exploration rate.<br>
$\zeta$ - exploration decay rate.<br>

<div class=" alert alert-warning">
    
### Student Task A5.5

Implement the `calc_exploration_rate()` function which calculates and returns the exploration rate using the above formula.
</div>

In [16]:
# Function that calculate the exploration rate for a given episode
def calc_exploration_rate(episode, min_exploration_rate, max_exploration_rate, exploration_decay_rate):
    ## Calculate the exploration rate
    # exploration_rate = ...
    
    # YOUR CODE HERE
    exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate) * math.e**(exploration_decay_rate * episode)
    
    
    
    return exploration_rate


In [17]:
# this cell is for tests


<div class=" alert alert-warning">
    
### Student Task A5.6

Next we will make the function for choosing an action given a state. This function includes an *exploration rate threshold* which is a randomly drawn number between 0 and 1. For every action, this thershold makes the agent perform an exploration action with a probability equal to the given *exploration rate*.
<br>
<strong><i><u>Your task<i></u></strong> is to complete the below function `choose_action()`. This function takes as input the Q-table, current state $s_{t}$, and exploration rate $\epsilon$.
</div>

In [18]:
def choose_action(q_table, state, exploration_rate):
    
    ## 1. Pick a random number between 0 and 1
    # exploration_rate_threshold = ...
    exploration_rate_threshold = np.random.uniform(0,1)
    
    ## 2. Exploration vs exploitation
    if exploration_rate > exploration_rate_threshold or sum(q_table[state, :]) == 0:
        ## Exploration, i.e., random action 
        # action = ...
        action = np.random.randint(action_space_size)
    else:
        ## Exploitation, i.e., perform the optimal action at this state according to the Q-table
        # action = ...
        action = np.argmax(q_table[state])
    
    
    # YOUR CODE HERE

    return action

In [19]:
# this cell is for tests


## Q-Learning Algorithm
Now it is time to train our agent using the Q-learning algorithm as shown below.
If everything works as intended, the below code should print some statistics from the training.

In [20]:
# Number of episodes to train
num_episodes = 10000

# Max number of steps before ending the round
max_steps_per_episode = 100

In [21]:
# Q-learning function
def train_q_learn(env, q_table):
    
    # Q-Learning algorithm
    rewards_all_episodes = []
    steps_all_episodes = []
    exploration_rate = max_exploration_rate

    # Play the game for n episodes
    for episode in range(num_episodes):
        state = env.reset()
        done = False
        rewards_current_episode = 0
        steps_current_episode = 0

        for step in range(max_steps_per_episode):
            # Chooce an action. Exploration vs exploitation trade-off with the epsilon greedy strategy
            action = choose_action(q_table, state, exploration_rate)

            # Perform the chosen action
            new_state, reward, done, info = env.step(action)

            # Upadete the Q-table
            q_table[state, action] = q_update(q_table, state, action, new_state, reward, learning_rate, discount_factor)

            # Set the new state
            state = new_state

            # Collect some statistics
            rewards_current_episode += reward
            steps_current_episode += 1

            if done == True:
                break

        # Update the exploration rate
        exploration_rate = calc_exploration_rate(episode, min_exploration_rate, max_exploration_rate, exploration_decay_rate)

        rewards_all_episodes.append(rewards_current_episode)
        steps_all_episodes.append(steps_current_episode)


    # Print the average rewards
    rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes), num_episodes/1000)
    steps_per_thousand_episodes = np.split(np.array(steps_all_episodes), num_episodes/1000)

    print('Training results, average reward and steps per thousand episodes:')
    count = 1000
    for i, _ in enumerate(rewards_per_thousand_episodes):
        r = rewards_per_thousand_episodes[i]
        s = steps_per_thousand_episodes[i]
        print('{}: rewards {:.4}, steps {:.4}'.format(count, sum(r/1000), sum(s/1000)))
        count += 1000
    return q_table

In [22]:
# Run the train_q_learn function
q_table = train_q_learn(env, q_table)

Training results, average reward and steps per thousand episodes:
1000: rewards 0.012, steps 7.841
2000: rewards 0.021, steps 7.597
3000: rewards 0.005, steps 7.577
4000: rewards 0.015, steps 7.868
5000: rewards 0.019, steps 7.933
6000: rewards 0.011, steps 7.755
7000: rewards 0.012, steps 7.648
8000: rewards 0.012, steps 7.59
9000: rewards 0.013, steps 7.809
10000: rewards 0.01, steps 7.609


In [23]:
print('Trained Q-table:')
print(q_table)

Trained Q-table:
[[0.94147726 0.95098705 0.95098719 0.94147726]
 [0.9414772  0.         0.96059318 0.95098712]
 [0.95098695 0.97029623 0.95098649 0.96059312]
 [0.96059283 0.         0.95098576 0.95098594]
 [0.95098698 0.96059305 0.         0.94147722]
 [0.         0.         0.         0.        ]
 [0.         0.98009739 0.         0.96059249]
 [0.         0.         0.         0.        ]
 [0.96059288 0.         0.97029613 0.95098679]
 [0.9605921  0.98009443 0.98009737 0.        ]
 [0.97029289 0.98999812 0.         0.97029225]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.98008433 0.98999561 0.97029099]
 [0.98006587 0.98999304 0.99999887 0.98007861]
 [0.         0.         0.         0.        ]]


## Playing Frozen Lake
Finally we will let the agent play Frozen Lake using the Q-table that we have learned above.

<div class=" alert alert-warning">
    
### Student Task A5.7

Now we are ready to put the agent to navigate the frozen lake based on what it has learned. By simply looking at the Q-table, maybe you can already see what route the agent will be taking?<br><br>
<strong><i><u>Your task<i></u></strong> is to run the below code and observe how the agent is navigating the frozen lake. Observe the learned Q-table to understand why the agent is taking this specific path.
</div>

In [24]:
def run_check():
    nbgrader_exec_env = os.environ.get('NBGRADER_EXECUTION')
    if nbgrader_exec_env is not None and nbgrader_exec_env in ['autograde', 'validate']:
        return False
    return True

# Defining the run_agent function
def run_agent(env, q_table):
    env.reset()
    env.render()
    if run_check():
        clear_output(wait=False)
        # Watch the agent play for a few rounds using the learned Q-table
        n_rounds = 3
        for episode in range(n_rounds):
            state = env.reset()
            done = False

            print('༺❁ EPISODE {} ❁༻'.format(episode+1))
            time.sleep(1)

            for step in range(max_steps_per_episode + 1):
                clear_output(wait=True)
                env.render()
                time.sleep(0.3)

                action = np.argmax(q_table[state, :])

                new_state, reward, done, info = env.step(action)

                if done:
                    clear_output(wait=True)
                    env.render()
                    if reward == 1:
                        print('༺❁ You reached the goal after {} steps! ❁༻'.format(step + 1))
                        time.sleep(3)
                    elif step == max_steps_per_episode - 1:
                        print('༺❁ No steps left ({} steps max)! ❁༻'.format(step + 1))
                        time.sleep(3)
                    else:
                        print('༺❁ You fell through a hole after {} steps! ❁༻'.format(step + 1))
                        time.sleep(3)
                    clear_output(wait=True)
                    break
                elif step == max_steps_per_episode - 1:
                    print('༺❁ No steps left ({} steps max)! ❁༻'.format(step + 1))
                    time.sleep(3)
                    clear_output(wait=True)
                    break
                state = new_state
        time.sleep(3)
        print('༺❁ Done ❁༻')
        env.close()

In [25]:
# Run the run_agent function with the trained Q-table
run_agent(env, q_table)

༺❁ Done ❁༻


## The Ice Can Get Slippery!
![Slippery ice](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS7iaMa5OcNHtJFKB1fZo_EvG-nQyPPTm_fsA&usqp=CAU "Slippery ice")

So far the ice has been easy to walk on, and we have seen that the agent has not had any problem in finding an optimal path leading to the goal. Now, however, the weather has changed and the ice has suddenly become very slippery.
In the next part of this exercise we have defined the frozen lake as:<br>
`env_frozen = gym.make('FrozenLake-v1', is_slippery=False, map_name="4x4")`<br>
`is_slippery=True` means that there is only a 1/3 chance that the agent will move in the intended 
direction, and 2/3 chance to move in one of the perpendicular directions (1/3 and 1/3). 
E.g., if **action = left**, then:
- P(move left) = 1/3
- P(move up) = 1/3
- P(move down) = 1/3

<div class=" alert alert-warning">
    
### Student Task A5.8a

First run the Q-Learning algorithm below and observe how the new trained Q-table looks like, and how the new statistics has become. Also observe how the behaviour of the agent has changed at run time.
    
### Student Task A5.8b
Next we want you to briefly explore the impact of different values assigned to the different parameters, and try to find the most promising values that optimize the reward gained by the agent (look at the scores from the last 1000 episodes).
The parameters in question are:
- learning_rate ($\alpha$)
- discount_factor ($\gamma$)
- max_exploration_rate ($\epsilon_{max}$)
- min_exploration_rate ($\epsilon_{min}$)
- exploration_decay_rate ($\zeta$)

<u>Be prepared to discuss your findings with your colleges and/or teaching assistants.</u><br>
Some additional things to think about: Why is the preferred first move of the agent to move left? Can you think of some possible approaches and strategies that could improve the performance of the agent further? Can you think of some completely different ways for how to solve this game/problem?
</div>

In [26]:
# Initiate the FrozenLake game with slippery ice
env_slippery = gym.make('FrozenLake-v1', is_slippery=True, map_name="4x4")


## Parameters used by the Q-training algorithm

# Learning rate (value between 0 and 1)
learning_rate = 0.1

# Reward discount used in the Bellman equation (value between 0 and 1)
discount_factor = 0.99

# Upper bound for the exploration rate (value between 0 and 1)
max_exploration_rate = 1.0

# Lower bound for the exploration rate (value between 0 and 1)
min_exploration_rate = 0.01

# How much the exploration rate decays per episode (down to the min_exploration_rate, value between 0 and 1).
exploration_decay_rate = 0.001


# Train the agent to navigate the slippery ice
q_table_slippery = np.zeros((env.observation_space.n, env.action_space.n))
if run_check():
    q_table_slippery = train_q_learn(env_slippery, q_table_slippery)

Training results, average reward and steps per thousand episodes:
1000: rewards 0.015, steps 7.648
2000: rewards 0.015, steps 7.55
3000: rewards 0.009, steps 7.537
4000: rewards 0.019, steps 7.596
5000: rewards 0.015, steps 7.724
6000: rewards 0.015, steps 7.743
7000: rewards 0.014, steps 7.471
8000: rewards 0.007, steps 7.76
9000: rewards 0.015, steps 7.522
10000: rewards 0.015, steps 7.738


In [27]:
print('Trained slippery Q-table:')
print(q_table_slippery)

Trained slippery Q-table:
[[0.51646879 0.51653112 0.50872616 0.50233627]
 [0.40688151 0.38485277 0.34629381 0.48667701]
 [0.4145584  0.39633905 0.38186804 0.45293238]
 [0.29797003 0.34442216 0.24165784 0.43306961]
 [0.53024309 0.43989703 0.43210604 0.37274891]
 [0.         0.         0.         0.        ]
 [0.27682202 0.1571603  0.31321639 0.06498009]
 [0.         0.         0.         0.        ]
 [0.40367613 0.434816   0.28110781 0.56545651]
 [0.40727074 0.61708435 0.53455316 0.39460015]
 [0.61544259 0.47398076 0.41065821 0.27732077]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.36482514 0.42328901 0.72401181 0.66719119]
 [0.71681058 0.85699216 0.82126738 0.79327832]
 [0.         0.         0.         0.        ]]


In [ ]:
# Run the agent to see how the agent now navigates the slippery ice.
run_agent(env_slippery, q_table_slippery)

  (Up)
SFFF
FHFH
FFFH
HFFG
